In [4]:
import win32com.client
from win32com.client import constants

def compare_word_documents(doc1_path, doc2_path):
    # Create a new instance of the Word application
    word_app = win32com.client.Dispatch("Word.Application")
    word_app.Visible = True  # Make Word visible (optional)

    # Open both documents
    doc1 = word_app.Documents.Open(doc1_path)
    doc2 = word_app.Documents.Open(doc2_path)

    # Compare the two documents
    compared_doc = word_app.CompareDocuments(doc1, doc2)

    # Access the revisions in the compared document
    revisions = compared_doc.Revisions
    for revision in revisions:
        if revision.Type == 1:
            print(revision.Type)
            print("Inserted text:")
            print(f"  - Text: {revision.Range.Text.strip()}")
            print(f"  - Cursor point: {revision.Range.Start}")
            print(f"  - Line: {revision.Range.Information(10)}")
            print(f"  - Property: {revision.Author}")
            print(f" -  Property: {revision.Date}")
        elif revision.Type == 2:
            print("Deleted text:")
            print(f"  - Text: {revision.Range.Text.strip()}")
            print(f"  - Cursor point: {revision.Range.Start}")
            print(f"  - Line: {revision.Range.Information(10)}")
            print(f"  - Property: {revision.Author}")
            print(f" -  Property: {revision.Date}")
        elif revision.Type == 3:
            print("Formatting change:")
            print(f"  - Cursor point: {revision.Range.Start}")
            print(f"  - Line: {revision.Range.Information(10)}")
            print(f"  - Property: {revision.Author}")
            print(f" -  Property: {revision.Date}")

    # Close the compared document without saving changes
    compared_doc.Close(False)

    # Close the original documents
    doc1.Close(False)
    doc2.Close(False)
    
    

# Example usage
compare_word_documents("D:\MCA22\Interviews\document2.docx", "D:\MCA22\Interviews\document3.docx")


Formatting change:
  - Cursor point: 2844
  - Line: 48
  - Property: Urvi Dedhia
 -  Property: 2024-02-20 09:33:00+00:00
Deleted text:
  - Text: NPTEL – Python For Data Science
  - Cursor point: 2845
  - Line: 49
  - Property: Urvi Dedhia
 -  Property: 2024-02-20 09:33:00+00:00


In [12]:
import win32com.client
from win32com.client import constants
print(constants)

In [12]:
import win32com.client
import datetime

def add_revision_details_to_document(doc1_path, doc2_path, output_doc_path):
    # Create a new instance of the Word application
    word_app = win32com.client.Dispatch("Word.Application")
    word_app.Visible = True  # Make Word visible (optional)

    # Open both documents
    doc1 = word_app.Documents.Open(doc1_path)
    doc2 = word_app.Documents.Open(doc2_path)

    # Compare the two documents
    compared_doc = word_app.CompareDocuments(doc1, doc2)

    # Access the revisions in the compared document
    revisions = compared_doc.Revisions
    compared_range = compared_doc.Range()
    print(compared_range)
    for revision in revisions:
        # Get the details to be added to the document
        revision_details = ""
        if revision.Type == 1:  # wdRevisionInsert
            revision_details = f"\nInserted text: {revision.Range.Text.strip()}\n"
        elif revision.Type == 2:  # wdRevisionDelete
            revision_details = f"\nDeleted text: {revision.Range.Text.strip()}\n"
        else:
            continue

        # Append cursor point and line number
        revision_details += f"Cursor point: {revision.Range.Start}"
        revision_details += f", Line: {revision.Range.Information(10)}"  # wdFirstCharacterLineNumber

        # Append author and date (if available)
        if hasattr(revision, 'Author'):
            revision_details += f", Author: {revision.Author}"
        if hasattr(revision, 'Property') and hasattr(revision.Property, 'Date'):
            date = revision.Property.Date
            date_only = datetime.datetime(date.year, date.month, date.day)
            revision_details += f", Date: {date.year}-{date.month}-{date.day}\n"

        # Insert revision details into the document
        compared_range.InsertAfter(revision_details)

     # Save the modified document
    compared_doc.SaveAs(output_doc_path)
    
    compared_doc.Close(False)
    # Close documents
    doc1.Close(False)
    doc2.Close(False)

    # Quit Word application
    word_app.Quit()

# Example usage
add_revision_details_to_document("D:\MCA22\Interviews\document2.docx", 
                                  "D:\MCA22\Interviews\document3.docx")

The Complete Python Bootcamp From Zero to Hero in Python. 


In [21]:
import win32com.client
import datetime
import json

def add_revision_details_to_document(doc1_path, doc2_path):
    # Create a new instance of the Word application
    word_app = win32com.client.Dispatch("Word.Application")
    word_app.Visible = True  # Make Word visible (optional)

    # Open both documents
    doc1 = word_app.Documents.Open(doc1_path)
    doc2 = word_app.Documents.Open(doc2_path)

    # Compare the two documents
    compared_doc = word_app.CompareDocuments(doc1, doc2)
    
    compared_doc.SaveAs("D:\MCA22\Interviews\document4.docx")
    obj = {
        "identical": False,
    }
    count=1
    allRevisions = {}

    # Access the revisions in the compared document
    revisions = compared_doc.Revisions
    for revision in revisions:
        # Get the range of the revision
        revision_range = revision.Range

        # Get the details to be added to the document
        revision_details = ""
        if revision.Type == 1:  # wdRevisionInsert
            revision_details = f"Inserted text: {revision.Range.Text.strip()}; "
        elif revision.Type == 2:  # wdRevisionDelete
            revision_details = f"Deleted text: {revision.Range.Text.strip()}; "
        else:
            continue

        # Append cursor point and line number
        revision_details += f"Cursor point: {revision.Range.Start}; "
        revision_details += f"Line: {revision.Range.Information(10)}"  # wdFirstCharacterLineNumber

        # Append author and date (if available)
        if hasattr(revision, 'Author'):
            revision_details += f"; Author: {revision.Author}"
        if hasattr(revision, 'Property') and hasattr(revision.Property, 'Date'):
            date = revision.Property.Date
            date_only = datetime.datetime(date.year, date.month, date.day)
            revision_details += f"; Date: {date.year}-{date.month}-{date.day}"

        # Insert revision details into the document immediately after the revision
        #revision_range.InsertAfter("\n"+revision_details+"\n")
        allRevisions[str(count)] = revision_details
        count+=1
    json_obj = json.dumps(allRevisions)
    obj["allRevisions"] = allRevisions
    comparedRange = compared_doc.Range()
    comparedRange.InsertAfter(json_obj)
    json_obj2 = json.dumps(obj)
    print(json_obj2)

    # Close documents
    doc1.Close(False)
    doc2.Close(False)

    # Quit Word application
    

# Example usage
add_revision_details_to_document("D:\MCA22\Interviews\document2.docx", 
                                  "D:\MCA22\Interviews\document3.docx")


{"identical": false, "allRevisions": {"1": "Inserted text: This is the added line.; Cursor point: 1966; Line: 35; Author: Urvi Dedhia", "2": "Deleted text: NPTEL \u2013 Python For Data Science; Cursor point: 2869; Line: 49; Author: Urvi Dedhia"}}


In [29]:
import win32com.client
import datetime

def add_revision_details_to_document(doc1_path, doc2_path):
    # Create a new instance of the Word application
    word_app = win32com.client.Dispatch("Word.Application")
    word_app.Visible = True  # Make Word visible (optional)

    # Open both documents
    doc1 = word_app.Documents.Open(doc1_path)
    doc2 = word_app.Documents.Open(doc2_path)

    # Compare the two documents
    compared_doc = word_app.CompareDocuments(doc1, doc2)

    # Initialize a list to store revision details dictionaries
    all_revision_details = []

    # Access the revisions in the compared document
    revisions = compared_doc.Revisions
    for revision in revisions:
        # Create a dictionary to store revision details
        revision_details = {}

        # Get the range of the revision
        revision_range = revision.Range

        # Get the details to be added to the dictionary
        if revision.Type == 1:  # wdRevisionInsert
            revision_details["Type"] = "Inserted text"
            revision_details["Text"] = revision.Range.Text.strip()
        elif revision.Type == 2:  # wdRevisionDelete
            revision_details["Type"] = "Deleted text"
            revision_details["Text"] = revision.Range.Text.strip()
        else:
            continue

        # Add cursor point and line number to the dictionary
        revision_details["Cursor point"] = revision.Range.Start
        revision_details["Line"] = revision.Range.Information(10)  # wdFirstCharacterLineNumber

        # Add author and date (if available) to the dictionary
        if hasattr(revision, 'Author'):
            revision_details["Author"] = revision.Author
        if hasattr(revision, 'Property') and hasattr(revision.Property, 'Date'):
            date = revision.Property.Date
            date_only = datetime.datetime(date.year, date.month, date.day)
            revision_details["Date"] = date.year+"-"+date.month+"-"+date.day

        # Append the revision details dictionary to the list
        all_revision_details.append(revision_details)
        revision_range.InsertAfter("\n")
        revision_range.InsertAfter(revision_details)

    # Save the modified document

    # Close documents
    #compared_doc.Close(False)
    doc1.Close(False)
    doc2.Close(False)

    # Quit Word application
    #word_app.Quit()

    # Return the list of all revision details dictionaries
    return all_revision_details

# Example usage
all_revision_details = add_revision_details_to_document("D:\MCA22\Interviews\document2.docx", 
                                                        "D:\MCA22\Interviews\document3.docx")

# Print all revision details dictionaries
for index, details in enumerate(all_revision_details, start=1):
    print(f"Revision {index}:")
    for key, value in details.items():
        print(f"  {key}: {value}")


Revision 1:
  Type: Inserted text
  Text: This is the added line.
  Cursor point: 1966
  Line: 35
  Author: Urvi Dedhia
Revision 2:
  Type: Deleted text
  Text: NPTEL – Python For Data Science
  Cursor point: 2989
  Line: 1
  Author: Urvi Dedhia


Comparing CSV File

In [12]:
import csv
from openpyxl import Workbook
import datetime

def read_csv(file_path):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        return list(csv.reader(csvfile))

def compare_csv(doc1_path, doc2_path):
    doc1 = read_csv(doc1_path)
    doc2 = read_csv(doc2_path)

    max_len = max(len(doc1), len(doc2))
    revisions = []

    for i in range(max_len):
        row1 = doc1[i] if i < len(doc1) else []
        row2 = doc2[i] if i < len(doc2) else []

        max_col_len = max(len(row1), len(row2))
        for j in range(max_col_len):
            cell1 = row1[j] if j < len(row1) else ""
            cell2 = row2[j] if j < len(row2) else ""

            if cell1 != cell2:
                revision_details = {
                    "Type": "Inserted text" if not cell1 else "Deleted text" if not cell2 else "Modified text",
                    "Text": f"Original: {cell1}, New: {cell2}",
                    "Row": i + 1,
                    "Column": j + 1
                }
                revisions.append(revision_details)
    return revisions

def save_revisions_to_excel(revisions, output_path):
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "Revision Details"

    headers = ["Type", "Text", "Row", "Column"]
    sheet.append(headers)

    for revision in revisions:
        row = [revision["Type"], revision["Text"], revision["Row"], revision["Column"]]
        sheet.append(row)

    workbook.save(output_path)

if __name__ == "__main__":
    doc1_path = "D:\MCA22\Interviews\csv2.csv"
    doc2_path = "D:\MCA22\Interviews\csv3.csv"
    output_path = "D:\MCA22\Interviews\csv4.csv"

    revisions = compare_csv(doc1_path, doc2_path)
    save_revisions_to_excel(revisions, output_path)

    print(f"Excel file generated: {output_path}")


Excel file generated: D:\MCA22\Interviews\csv4.csv


In [16]:
import win32com.client as win32
import os
import csv

def read_csv(file_path):
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        return list(csv.reader(csvfile))

def compare_csv(doc1_path, doc2_path):
    doc1 = read_csv(doc1_path)
    doc2 = read_csv(doc2_path)

    max_len = max(len(doc1), len(doc2))
    revisions = []

    for i in range(max_len):
        row1 = doc1[i] if i < len(doc1) else []
        row2 = doc2[i] if i < len(doc2) else []

        max_col_len = max(len(row1), len(row2))
        for j in range(max_col_len):
            cell1 = row1[j] if j < len(row1) else ""
            cell2 = row2[j] if j < len(row2) else ""

            if cell1 != cell2:
                revision_details = {
                    "Type": "Inserted text" if not cell1 else "Deleted text" if not cell2 else "Modified text",
                    "Text": f"Original: {cell1}, New: {cell2}",
                    "Row": i + 1,
                    "Column": j + 1
                }
                revisions.append(revision_details)
    return revisions

def save_revisions_to_excel(revisions, output_path):
    excel = win32.Dispatch('Excel.Application')
    wb = excel.Workbooks.Add()
    sheet = wb.ActiveSheet
    sheet.Name = "Revision Details"

    headers = ["Type", "Text", "Row", "Column"]
    for col_num, header in enumerate(headers, 1):
        sheet.Cells(1, col_num).Value = header

    for row_num, revision in enumerate(revisions, 2):
        sheet.Cells(row_num, 1).Value = revision["Type"]
        sheet.Cells(row_num, 2).Value = revision["Text"]
        sheet.Cells(row_num, 3).Value = revision["Row"]
        sheet.Cells(row_num, 4).Value = revision["Column"]

    wb.SaveAs(output_path)
    wb.Close()
    excel.Quit()

if __name__ == "__main__":
    doc1_path = "D:\MCA22\Interviews\csv2.csv"
    doc2_path = "D:\MCA22\Interviews\csv3.csv"
    output_path = "D:\MCA22\Interviews\csv4.csv"

    revisions = compare_csv(doc1_path, doc2_path)
    save_revisions_to_excel(revisions, output_path)

    print(f"Excel file generated: {output_path}")


Excel file generated: D:\MCA22\Interviews\csv4.csv
